In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

red_wine_path= ("../input/red-wine-quality-cortez-et-al-2009/winequality-red.csv")
red_wine=pd.read_csv(red_wine_path)

red_wine.head()
#red_wine.describe()




In [ ]:
wine_columns= red_wine.columns
print(wine_columns)

#check dataset for missing values
missing_values= [i for i in red_wine.columns if red_wine[i].isnull().any()]

# eine leere Liste gibt den Wert false, der wieder negiert wird wodurch die Bedingnug erfüllt würde
if not missing_values:
    print("Keine fehlenden Werte")
    
cols_with_object= red_wine.dtypes == "object"

if not list(cols_with_object[cols_with_object].index): #Notiz zum recherchieren: er nimmt bei cols with object[object] nur die Werte mit true, aber wie genau?
    print("Keine kateogirschen Werte")


In [ ]:
#target, features einreichten und Datensatz splitten
from sklearn.model_selection import train_test_split

#features: alles außer quality
X=red_wine.copy()
y=X.pop("quality")

#oder:
#red_wine.head()

#X= red_wine[red_wine.columns[0:11]]
#y=red_wine["quality"]

X_train, X_valid, y_train, y_valid = train_test_split (X,y, test_size=0.2, random_state=0)
print("train_test_split done")

In [ ]:
#Trainingsdatensatz für das Modell Standartisieren
from sklearn.preprocessing import StandardScaler

sc= StandardScaler()
X_train= sc.fit_transform(X_train)
X_valid= sc.transform(X_valid)

input_shape= [X_train.shape[1]]

In [ ]:
#model definieren
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

regression_model = keras.Sequential([
    layers.Dense(1500, activation ="tanh", input_shape=input_shape), #1500
    layers.Dense(64, activation ="tanh"), #64 mit tanh hat auch gut verbessert um 0.15 /der wert 0.15 erhielt ich bei "relu" und 250 units
    layers.Dense(64, activation ="relu"), #64
    layers.Dense(1) #Notiz: Warum braucht man hier gar keine activaton Function im Vergleich zu Softmax --> Softmax rechnet einen Zahlenvector in Wahrscheinlichkeiten, aber wie funktioniert das?
])

In [ ]:
#model compilieren

regression_model.compile(
    optimizer="adam", #adam oder rmsprop
    loss="mae", #mae oder mse
    #metrics="accuracy" hier nicht brauchbar, da nur für classification
   metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [ ]:
history= regression_model.fit(
    X_train, y_train,
    batch_size=64, #105
    epochs=20, #20
    #verbose=0, #turn of training log
    
)

In [ ]:
import pandas as pd

history_df = pd.DataFrame(history.history)
# Start the plot at epoch 5. You can change this to get a different view.
history_df.loc[5:, ['loss']].plot();

In [ ]:
y_pred= regression_model.predict(X_valid)

In [ ]:
if len(y_pred) == len(y_pred):
    len_y_pred=len(y_pred)
else:
    print("Anzahl der Instanzen, die dem Model zum validieren in X_valid übergeben werden stimmt nicht mit den zu überprüfenden Instanzen in y_valid ")
y_pred_list=[]
for b in range(len_y_pred):
    
    y_pred_list.append(y_pred[b][0])


#print(y_pred_list)

In [ ]:
y_valid_list= y_valid.values.tolist()
#print(y_valid_list)

In [ ]:
for i in range(20):
    print("Predicted %s ---> Expected %d"%(y_pred_list[i],y_valid_list[i]))

In [ ]:
from sklearn.metrics import r2_score
print("Der r2_Score beträgt:",r2_score(y_valid_list, y_pred_list))
print("Der r2_Score beträgt:",r2_score(y_valid, y_pred))
print(X_valid)

In [ ]:
import shap
shap.initjs()

# rather than use the whole training set to estimate expected values, we summarize with
# a set of weighted kmeans, each weighted by the number of points they represent.
X_train_summary = shap.kmeans(X_train, 10)

explainer = shap.KernelExplainer(regression_model.predict, X_train_summary)
shap_values = explainer.shap_values(X_valid)
shap.summary_plot(shap_values, X_valid)


print("DONE")
